In [ ]:
# import arcpy to access arcgis tools
import arcpy

# import os to read files
import os

# create empty array
array_features = []

# create scratch geodatabase
scratch_GDB = arcpy.env.scratchGDB + os.path.sep

# USER MUST CHANGE THIS TO THEIR FILE LOCATION
# set pathname to the location of the folder that contains ERO NetCDF files 
path_name = r'C:\Users\meira\OneDrive - University of Florida\UF_Thesis_Research\ERO_NetCDF\spr2'

In [ ]:
# create loop to read through all files in above folder
for filename in os.listdir(path_name):
    
    # set variables for 'Make NetCDF Feature Layer' tool
    in_netCDF_file = path_name + '\\' + filename   
    in_variable_ERO = 'FCST_ERO_Surface_FULL'
    in_variable_OBS = 'OBS_ALL_A365324_FULL_MAX_49'
    in_x_variable = 'lon'
    in_y_variable = 'lat'
    out_feature_layer_ERO = 'ERO' + '_' + filename[19:27]
    out_feature_layer_OBS = 'OBS' + '_' + filename[19:27]
    row_dimensions = 'lon', 'lat'

    # make two feature layers: one for EROs, one for observations/proxies (OBS)
    ERO_FL = arcpy.MakeNetCDFFeatureLayer_md(in_netCDF_file, in_variable_ERO, 
                                             in_x_variable, in_y_variable, 
                                             out_feature_layer_ERO, row_dimensions)

    OBS_FL = arcpy.MakeNetCDFFeatureLayer_md(in_netCDF_file, in_variable_OBS, 
                                             in_x_variable, in_y_variable, 
                                             out_feature_layer_OBS, row_dimensions)
    
    # select non-zero values from both feature layers (zeros are no ERO/observation/proxy)
    ERO_no_zeros = arcpy.SelectLayerByAttribute_management(
        ERO_FL, 'NEW_SELECTION', '"FCST_ERO_Surface_FULL" <> 0')

    OBS_no_zeros = arcpy.SelectLayerByAttribute_management(
        OBS_FL, 'NEW_SELECTION', '"OBS_ALL_A365324_FULL_MAX_49" <> 0')
    
    # send ERO + observation/proxy feature layers without zeros to new feature layer
    # store in scratch geodatabase
    ERO_FL_no_zeros = arcpy.FeatureClassToFeatureClass_conversion(
        ERO_no_zeros, scratch_GDB, out_feature_layer_ERO)

    OBS_FL_no_zeros = arcpy.FeatureClassToFeatureClass_conversion(
        OBS_no_zeros, scratch_GDB, out_feature_layer_OBS)
    
    # create location in scratch geodatabse for EROs erased from observations/proxies
    erased_name = scratch_GDB + "OBS_ERO_erased_" + filename[19:27]
    
    # erase ERO layer form observation/proxy layer
    # output contains only observations/proxies that were not captured by EROs
    erase_output = arcpy.Erase_analysis(OBS_FL_no_zeros, ERO_FL_no_zeros, erased_name)
    
    # delete ERO and observation/proxy layer without zeros from scratch geodatabase
    arcpy.Delete_management(OBS_FL_no_zeros)
    arcpy.Delete_management(ERO_FL_no_zeros)
    
    # add erased layer to array
    array_features.append(erase_output)

In [ ]:
# count overlapping points in all feature layers
# arcpy.CountOverlappingFeatures_analysis(array_features, 'overlap')

# delete scratch geodatabase
arcpy.Delete_management(scratch_GDB)